In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
print(os.listdir("/home/sunilvikram/data"))

['xgboost_trainsamplediffparams.pkl', 'xgboost_4c_withmerge.pkl', 'mnt', 'sample_submission.csv.zip', 'test_supplement.csv.zip', 'nohup.out', '.ipynb_checkpoints', 'submission.csv', 'EDAReport.ipynb', 'train_sample.csv.zip', 'xgboost_4c_withmergediffparams.pkl', 'BuildModel_merge_diffparams.ipynb', 'Dalon_notebook.ipynb', 'submission_4c_withmany.csv', 'test_withmerge.csv', 'xgbts_merge_withdiffparams.csv', 'PlotImportance_withmerge.ipynb', 'xgboost_8c.pkl', 'Untitled.ipynb', 'sample_submission.csv', 'BuildModel_merge_8c.ipynb', 'xgboost_4c_withmany.pkl', 'test.csv', 'train.csv.zip', 'test.csv.zip', 'BuildModel.ipynb', 'submission_4c_withmerge.csv', 'xgb_merge_withdiffparams.csv']


In [3]:
usefulcolumns=['ip','app','device','os', 'channel', 'click_time', 'is_attributed']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        }

In [4]:
traindata= pd.read_csv('mnt/ssd/kaggle-talkingdata2/competition_files/train.csv',parse_dates=['click_time'],dtype=dtypes,usecols=usefulcolumns)

In [5]:
traindata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 7 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
click_time       datetime64[ns]
is_attributed    uint8
dtypes: datetime64[ns](1), uint16(4), uint32(1), uint8(1)
memory usage: 3.6 GB


In [6]:
AttributedDF= traindata[traindata.is_attributed==1]

In [7]:
NotAttributed= traindata[traindata.is_attributed==0]

In [8]:
NotAttributed.shape

(184447044, 7)

In [9]:
NonAttrCount=AttributedDF.shape[0]*10 -AttributedDF.shape[0]

In [10]:
NonAttrCount

4111614

In [11]:
#NonAttrCount= NonAttrCount *10

In [12]:
#lines=NotAttributed.shape[0]
#howmanytoskip = lines-NonAttrCount

#skiplines = np.random.choice(np.arange(1, lines), size=NonAttrCount, replace=False)

#sort the list
#skiplines=np.sort(skiplines)

In [13]:
NotAttributed=NotAttributed.sample(n=NonAttrCount)

In [14]:
NotAttributed.shape

(4111614, 7)

In [15]:
newdf = [AttributedDF,NotAttributed]

In [16]:
newtraindata= pd.concat(newdf)

In [17]:
newtraindata.shape

(4568460, 7)

In [18]:
import gc
gc.collect()

32

In [19]:
del traindata


In [20]:
del NotAttributed
del AttributedDF
gc.collect()

0

In [21]:
testdata= pd.read_csv('test.csv',parse_dates=['click_time'],dtype=dtypes)

In [22]:
newtraindata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4568460 entries, 103 to 135319886
Data columns (total 7 columns):
ip               uint32
app              uint16
device           uint16
os               uint16
channel          uint16
click_time       datetime64[ns]
is_attributed    uint8
dtypes: datetime64[ns](1), uint16(4), uint32(1), uint8(1)
memory usage: 126.3 MB


In [23]:
testdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18790469 entries, 0 to 18790468
Data columns (total 7 columns):
click_id      int64
ip            uint32
app           uint16
device        uint16
os            uint16
channel       uint16
click_time    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), uint16(4), uint32(1)
memory usage: 501.8 MB


In [24]:
Y = newtraindata['is_attributed']

In [25]:
newtraindata.drop(['is_attributed'], axis=1, inplace=True)

In [26]:
testdata.drop(['click_id'], axis=1, inplace=True)
gc.collect()

55

In [27]:
nrow_train = newtraindata.shape[0]
merge = pd.concat([newtraindata, testdata])

In [28]:
merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23358929 entries, 103 to 18790468
Data columns (total 6 columns):
ip            uint32
app           uint16
device        uint16
os            uint16
channel       uint16
click_time    datetime64[ns]
dtypes: datetime64[ns](1), uint16(4), uint32(1)
memory usage: 623.8 MB


In [29]:
def timeFeature(df):
    df['dayofclick'] = df['click_time'].dt.day.astype('uint8')
    df['hourofclick'] = df['click_time'].dt.hour.astype('uint8')
    df['minofclick'] = df['click_time'].dt.minute.astype('uint8')
    df['secofclick'] = df['click_time'].dt.second.astype('uint8')
    return df

In [30]:
merge=timeFeature(merge)
merge.head()

,ip,app,device,os,channel,click_time,dayofclick,hourofclick,minofclick,secofclick
103,204158,35,1,13,21,2017-11-06 15:41:07,6,15,41,7
1504,29692,9,1,22,215,2017-11-06 16:00:02,6,16,0,2
1798,64516,35,1,13,21,2017-11-06 16:00:02,6,16,0,2
2102,172429,35,1,46,274,2017-11-06 16:00:03,6,16,0,3
3056,199085,35,1,13,274,2017-11-06 16:00:04,6,16,0,4


In [31]:
import gc
gc.collect()

201

In [32]:
# Define all the groupby transformations
def groupByFeatures(df):
    GROUPBY_AGGREGATIONS = [
    
    # V1 - GroupBy Features #
    #########################    
   
    # Count, for ip-day-hour
    {'groupby': ['ip','dayofclick','hourofclick'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app
    {'groupby': ['ip', 'app'], 'select': 'channel', 'agg': 'count'},        
    # Count, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'channel', 'agg': 'count'},
    # Count, for ip-app-day-hour
    {'groupby': ['ip','app','dayofclick','hourofclick'], 'select': 'channel', 'agg': 'count'},
    # Mean hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hourofclick', 'agg': 'mean'}, 
    
    # V2 - GroupBy Features #
    #########################
    # Average clicks on app by distinct users; is it an app they return to?
    {'groupby': ['app'], 
     'select': 'ip', 
     'agg': lambda x: float(len(x)) / len(x.unique()), 
     'agg_name': 'AvgViewPerDistinct'
    },
    # How popular is the app or channel?
    {'groupby': ['app'], 'select': 'channel', 'agg': 'count'},
    {'groupby': ['channel'], 'select': 'app', 'agg': 'count'}
    ]

# Apply all the groupby transformations
    for spec in GROUPBY_AGGREGATIONS:
    
    # Name of the aggregation we're applying
        agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']
    
    # Info
        print("Grouping by {}, and aggregating {} with {}".format(
        spec['groupby'], spec['select'], agg_name
        ))
    
    # Unique list of features to select
        all_features = list(set(spec['groupby'] + [spec['select']]))
    
    # Name of new feature
        new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])
    
    # Perform the groupby
        gp = df[all_features]. \
        groupby(spec['groupby'])[spec['select']]. \
        agg(spec['agg']). \
        reset_index(). \
        rename(index=str, columns={spec['select']: new_feature})
        
    # Merge back to X_train
        df=df.merge(gp, on=spec['groupby'], how='left')
    return df

In [33]:
merge=groupByFeatures(merge)

Grouping by ['ip', 'dayofclick', 'hourofclick'], and aggregating channel with count
Grouping by ['ip', 'app'], and aggregating channel with count
Grouping by ['ip', 'app', 'os'], and aggregating channel with count
Grouping by ['ip', 'app', 'dayofclick', 'hourofclick'], and aggregating channel with count
Grouping by ['ip', 'app', 'channel'], and aggregating hourofclick with mean
Grouping by ['app'], and aggregating ip with AvgViewPerDistinct
Grouping by ['app'], and aggregating channel with count
Grouping by ['channel'], and aggregating app with count


In [34]:
merge.head()

,ip,app,device,os,channel,click_time,dayofclick,hourofclick,minofclick,secofclick,ip_dayofclick_hourofclick_count_channel,ip_app_count_channel,ip_app_os_count_channel,ip_app_dayofclick_hourofclick_count_channel,ip_app_channel_mean_hourofclick,app_AvgViewPerDistinct_ip,app_count_channel,channel_count_app
0,204158,35,1,13,21,2017-11-06 15:41:07,6,15,41,7,1,1,1,1,15.00,2.228382,68057,70263
1,29692,9,1,22,215,2017-11-06 16:00:02,6,16,0,2,1,58,2,1,9.00,34.891723,3257910,282562
2,64516,35,1,13,21,2017-11-06 16:00:02,6,16,0,2,8,16,3,2,7.25,2.228382,68057,70263
3,172429,35,1,46,274,2017-11-06 16:00:03,6,16,0,3,1,2,1,1,16.00,2.228382,68057,27476
4,199085,35,1,13,274,2017-11-06 16:00:04,6,16,0,4,1,4,1,1,16.00,2.228382,68057,27476


In [35]:
import gc
gc.collect()

167

In [36]:
def timebtwClicks(df):
    GROUP_BY_NEXT_CLICKS = [
        {'groupby': ['ip']},
    {'groupby': ['ip', 'app']},
    {'groupby': ['ip', 'channel']},
    {'groupby': ['ip', 'os']},
    ]

# Calculate the time to next click for each group
    for spec in GROUP_BY_NEXT_CLICKS:
    
    # Name of new feature
        new_feature = '{}_nextClick'.format('_'.join(spec['groupby']))    
    
    # Unique list of features to select
        all_features = spec['groupby'] + ['click_time']
    
    # Run calculation
        print(">> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}")
        df[new_feature] = df[all_features].groupby(spec['groupby']).click_time.transform(lambda x: x.diff().shift(-1)).dt.second
    
    return df

In [37]:
merge = timebtwClicks(merge)

>> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}
>> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}
>> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}
>> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}


In [38]:
merge.head()

,ip,app,device,os,channel,click_time,dayofclick,hourofclick,minofclick,secofclick,...,ip_app_os_count_channel,ip_app_dayofclick_hourofclick_count_channel,ip_app_channel_mean_hourofclick,app_AvgViewPerDistinct_ip,app_count_channel,channel_count_app,ip_nextClick,ip_app_nextClick,ip_channel_nextClick,ip_os_nextClick
0,204158,35,1,13,21,2017-11-06 15:41:07,6,15,41,7,...,1,1,15.00,2.228382,68057,70263,17.0,NaN,NaN,17.0
1,29692,9,1,22,215,2017-11-06 16:00:02,6,16,0,2,...,2,1,9.00,34.891723,3257910,282562,4.0,46.0,30.0,16.0
2,64516,35,1,13,21,2017-11-06 16:00:02,6,16,0,2,...,3,2,7.25,2.228382,68057,70263,52.0,52.0,52.0,28.0
3,172429,35,1,46,274,2017-11-06 16:00:03,6,16,0,3,...,1,1,16.00,2.228382,68057,27476,25.0,25.0,NaN,NaN
4,199085,35,1,13,274,2017-11-06 16:00:04,6,16,0,4,...,1,1,16.00,2.228382,68057,27476,21.0,21.0,21.0,58.0


In [39]:
merge=merge.drop('click_time',axis=1)

In [40]:
merge.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'dayofclick', 'hourofclick',
       'minofclick', 'secofclick', 'ip_dayofclick_hourofclick_count_channel',
       'ip_app_count_channel', 'ip_app_os_count_channel',
       'ip_app_dayofclick_hourofclick_count_channel',
       'ip_app_channel_mean_hourofclick', 'app_AvgViewPerDistinct_ip',
       'app_count_channel', 'channel_count_app', 'ip_nextClick',
       'ip_app_nextClick', 'ip_channel_nextClick', 'ip_os_nextClick'],
      dtype='object')

In [41]:
train = merge[:nrow_train]
test = merge[nrow_train:]

In [42]:
del merge
gc.collect()

544

In [43]:
train.shape

(4568460, 21)

In [44]:
test.shape

(18790469, 21)

In [45]:
from sklearn.cross_validation import train_test_split 
import xgboost as xgb

In [46]:
params = {'eta': 0.1, # learning rate
          'tree_method': "auto", 
          'max_depth': 4, 
          'subsample': 0.8, 
          'colsample_bytree': 0.7, 
          'colsample_bylevel':0.7,
          'min_child_weight':0,
          'alpha':4,
          'objective': 'binary:logistic', 
          'scale_pos_weight':9,
          'eval_metric': 'auc', 
          'random_state': 99,
          #'threads': 8,
          'silent': True}
          
x1, x2, y1, y2 = train_test_split(train, Y, test_size=0.1)
del train
del Y

gc.collect()

dtrain = xgb.DMatrix(x1, y1)
del x1, y1
gc.collect()

dvalid = xgb.DMatrix(x2, y2)
del x2, y2
gc.collect()

watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
model = xgb.train(params, dtrain, 500, watchlist, maximize=True, early_stopping_rounds=50, verbose_eval=10)
del dtrain
del dvalid
gc.collect()


[0]	train-auc:0.941878	valid-auc:0.941516
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[10]	train-auc:0.956079	valid-auc:0.956244
[20]	train-auc:0.960342	valid-auc:0.960214
[30]	train-auc:0.963412	valid-auc:0.96325
[40]	train-auc:0.965731	valid-auc:0.96559
[50]	train-auc:0.966919	valid-auc:0.966716
[60]	train-auc:0.968078	valid-auc:0.967845
[70]	train-auc:0.968965	valid-auc:0.968701
[80]	train-auc:0.969688	valid-auc:0.969387
[90]	train-auc:0.970176	valid-auc:0.969856
[100]	train-auc:0.970596	valid-auc:0.970257
[110]	train-auc:0.970997	valid-auc:0.970617
[120]	train-auc:0.971443	valid-auc:0.971052
[130]	train-auc:0.971781	valid-auc:0.971376
[140]	train-auc:0.972164	valid-auc:0.971739
[150]	train-auc:0.972475	valid-auc:0.972028
[160]	train-auc:0.972675	valid-auc:0.972216
[170]	train-auc:0.972949	valid-auc:0.972439
[180]	train-auc:0.973163	valid-auc:0.972634
[190]	train-auc:0.973358	valid-auc

21

In [47]:
gc.collect()

0

In [48]:
import pickle

In [49]:
pickle.dump(model, open('xgboost_trainsamplediffparams.pkl', 'wb'))

In [50]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18790469 entries, 4568460 to 23358928
Data columns (total 21 columns):
ip                                             uint32
app                                            uint16
device                                         uint16
os                                             uint16
channel                                        uint16
dayofclick                                     uint8
hourofclick                                    uint8
minofclick                                     uint8
secofclick                                     uint8
ip_dayofclick_hourofclick_count_channel        int64
ip_app_count_channel                           int64
ip_app_os_count_channel                        int64
ip_app_dayofclick_hourofclick_count_channel    int64
ip_app_channel_mean_hourofclick                float64
app_AvgViewPerDistinct_ip                      float64
app_count_channel                              int64
channel_count_app     

In [51]:
#test.to_csv('test_withmerge.csv',index=False)

In [52]:
dtest = xgb.DMatrix(test)
del test
gc.collect()

33

In [53]:
testdata= pd.read_csv('test.csv',parse_dates=['click_time'],dtype=dtypes)
sub = pd.DataFrame()
sub['click_id'] = testdata['click_id']
sub['is_attributed'] = model.predict(dtest, ntree_limit=model.best_ntree_limit)
sub.to_csv('xgbts_merge_withdiffparams.csv',index=False)